In [10]:
import base64
import json
import boto3
from pathlib import Path

# -------------------------
# Config
# -------------------------
REGION = "eu-north-1"
ENDPOINT_NAME = "BMI-predcitor-v4"

runtime = boto3.client("sagemaker-runtime", region_name=REGION)

# -------------------------
# Helpers
# -------------------------
def to_b64(path: str) -> str:
    b64 = base64.b64encode(Path(path).read_bytes()).decode("utf-8")
    return "".join(b64.split())  # remove whitespace/newlines

def invoke_bmi(front_path: str, side_path: str, height_cm: float, gender: str):
    payload = {
        "height_cm": float(height_cm),
        "gender": gender,
        "front_image_b64": to_b64(front_path),
        "side_image_b64": to_b64(side_path),
    }

    resp = runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType="application/json",
        Accept="application/json",
        Body=json.dumps(payload).encode("utf-8"),
    )

    raw = resp["Body"].read().decode("utf-8")
    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        return {"raw_response": raw}

# -------------------------
# Run
# -------------------------
result = invoke_bmi(
    front_path="front.png",
    side_path="side.png",
    height_cm=166.2,
    gender="female",
)

print(json.dumps(result, indent=2))


{
  "ankle": 23.113271713256836,
  "arm-length": 48.6572151184082,
  "bicep": 23.996253967285156,
  "calf": 35.19050216674805,
  "chest": 95.56001281738281,
  "forearm": 23.735366821289062,
  "hip": 101.52625274658203,
  "leg-length": 80.11998748779297,
  "shoulder-breadth": 32.7743034362793,
  "shoulder-to-crotch": 61.426578521728516,
  "thigh": 55.21903991699219,
  "waist": 83.39549255371094,
  "wrist": 15.189276695251465,
  "weight_kg": 58.00539779663086
}


In [9]:
import json

# paste your model output here (or load it from a file)
out = {
  "ankle": 25.243059158325195,
  "arm-length": 49.7320556640625,
  "bicep": 32.270782470703125,
  "calf": 37.56290817260742,
  "chest": 103.24332427978516,
  "forearm": 27.999835968017578,
  "hip": 101.26437377929688,
  "leg-length": 78.0538558959961,
  "shoulder-breadth": 37.65117645263672,
  "shoulder-to-crotch": 68.9615249633789,
  "thigh": 54.85026168823242,
  "waist": 87.97280883789062,
  "wrist": 17.42530059814453,
  "weight_kg": 58.00539779663086
}

height_cm = 166.2  # your input height

weight_kg = float(out["weight_kg"])
height_m = float(height_cm) / 100.0

bmi = weight_kg / (height_m ** 2)

print("Weight (kg):", weight_kg)
print("Height (m):", height_m)
print("BMI:", round(bmi, 2))

# optional: include BMI in the output dict
out["bmi"] = bmi
print("\nOutput with BMI:")
print(json.dumps(out, indent=2))


Weight (kg): 40.99466323852539
Height (m): 1.5419999999999998
BMI: 17.24

Output with BMI:
{
  "ankle": 25.243059158325195,
  "arm-length": 49.7320556640625,
  "bicep": 32.270782470703125,
  "calf": 37.56290817260742,
  "chest": 103.24332427978516,
  "forearm": 27.999835968017578,
  "hip": 101.26437377929688,
  "leg-length": 78.0538558959961,
  "shoulder-breadth": 37.65117645263672,
  "shoulder-to-crotch": 68.9615249633789,
  "thigh": 54.85026168823242,
  "waist": 87.97280883789062,
  "wrist": 17.42530059814453,
  "weight_kg": 40.99466323852539,
  "bmi": 17.240846122039613
}
